In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from track import get_infected_img
from track import get_tracks, plot_tracks, get_events_df, get_laptrack
from tqdm import tqdm
import sys

sys.path.insert(0, '..') # in order to be able to import from scripts.py
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

# Analize the results

In [ ]:
df_results = pd.read_csv("results/results.csv")
df_results = df_results.drop(columns="Unnamed: 1")

In [ ]:
df_results[df_results['tree_id'] == 0]

In [ ]:
i = 301     # increase max_distance and then try i = 43
df = pd.read_csv(f"results/data/out-{i}.csv")
img = get_infected_img(df)
lt = get_laptrack(max_distance=25)
track_df, split_df, merge_df = get_tracks(img, lt)
events_df = get_events_df(track_df, split_df)
plot_tracks(track_df, split_df, merge_df)

# Get data

In [ ]:
sim_num = 500
folder = "results/data"

In [ ]:
for channel_height in [7]:

    client = VisAVisClient(
        visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    )

    for interval in [100]:

        protocol_file_path = make_protocol(
            pulse_intervals = [interval, 1500],
            duration=4,
            out_folder='./'
        )
        
        for sim in tqdm(range(sim_num)):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            result.states.to_csv(f"{folder}/out-{sim}.csv", index=False)

In [ ]:
events = []
lt = get_laptrack(max_distance = 32)

for i in tqdm(range(sim_num)):
    df = pd.read_csv(f"{folder}/out-{i}.csv")
    img = get_infected_img(df)
    track_df, split_df, merge_df = get_tracks(img, lt)
    if len(split_df) == 0:
        events.append(pd.DataFrame())
        continue
    events_df = get_events_df(track_df, split_df)
    events_df.index.name = 'event_id'
    events.append(events_df)

In [ ]:
df_results = pd.concat(events, names=['sim_id'], keys=range(sim_num))
df_results.to_csv(f"{folder}/results.csv")

In [ ]:
df_results['tree_id'].value_counts()